In [107]:
import pandas as pd
from scipy.stats import shapiro, levene, kruskal, wilcoxon
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pandas as pd
import numpy as np
from scipy import stats
import numpy as np

# Data

In [108]:
data = pd.read_csv("../data/processed/merged_features.csv").drop(["Unnamed: 0"], axis=1)
data


,Year,Average temperature (degrees C),Average wind Speed (km/h),Average visibility (km),Yearly Average of Daily Volume,BICYCLE COLLISIONS,MOTORCYCLE COLLISIONS,PEDESTRIAN COLLISIONS,Total Collisions,Total Fatalities,FATALITIES: INTERSECTION,FATALITIES: MIDBLOCK,INJURIES: INTERSECTION,INJURIES: MIDBLOCK,FATAL COLLISIONS,SERIOUS INJURY COLLISIONS,MINOR INJURY COLLISIONS,TOTAL SERIOUS & MINOR INJURY COLLISIONS,TOTAL FATAL & INJURY COLLISIONS,RATIO FATALITIES + INJURIES
0,2010,3.352347,11.053013,15.908633,NaN,182,211,306,28480,27,13,14,3314,1307,24,423,3345,3768,3792,0.133146
1,2011,3.445416,13.142069,16.600753,12193.134872,190,199,316,23442,22,12,9,3010,1256,22,389,3093,3482,3504,0.149475
2,2012,4.414667,12.638326,16.167048,11896.012759,177,157,296,23243,27,5,22,3003,1200,26,426,2937,3363,3389,0.145807
3,2013,2.410434,12.647422,18.062787,11975.502906,177,172,298,24805,23,11,12,2825,1127,23,375,2848,3223,3246,0.130861
4,2014,2.810970,12.625795,18.412174,12330.219780,177,163,319,24627,23,10,13,2515,1028,22,345,2567,2912,2934,0.119138
5,2015,4.941236,12.332064,18.046739,13160.213272,178,208,316,25516,32,18,13,2727,948,30,329,2704,3033,3063,0.120042
6,2016,4.812860,12.175093,17.538099,12688.032454,171,191,292,23139,22,11,10,2368,778,21,281,2375,2656,2677,0.115692
7,2017,3.731350,12.627638,17.347945,11896.841582,143,154,270,23905,27,13,13,2393,822,26,297,2413,2710,2736,0.114453
8,2018,1.785195,12.315178,17.535189,11064.122497,130,143,250,24003,19,8,11,2302,848,19,287,2323,2610,2629,0.109528
9,2019,2.479827,12.075129,18.848681,11392.131920,130,104,221,21943,14,8,6,1813,613,14,244,1822,2066,2080,0.094791


# Inference Testing I

### Do mean-collision for bicycle, motorcycle and pedestrian groups differ?

In [109]:
# Testing Normality and Equal Variance using Shpiro-Wilk and Levenes Test
_, p_sw_bike = shapiro(data["BICYCLE COLLISIONS"])
_, p_sw_moto = shapiro(data["MOTORCYCLE COLLISIONS"])
_, p_sw_ped  = shapiro(data["PEDESTRIAN COLLISIONS"])

_, p_levene = levene(data["BICYCLE COLLISIONS"], data["MOTORCYCLE COLLISIONS"], data["PEDESTRIAN COLLISIONS"])

print("P-Value for Shapiro Wilk - BIKE: " + str(p_sw_bike))
print("P-Value for Shapiro Wilk - MOTO: " + str(p_sw_moto))
print("P-Value for Shapiro Wilk - PEDESTRIAN: " + str(p_sw_ped))

print("\nP-Value for LEVENE: " + str(p_levene))


P-Value for Shapiro Wilk - BIKE: 0.05037878826260567
P-Value for Shapiro Wilk - MOTO: 0.6725215911865234
P-Value for Shapiro Wilk - PEDESTRIAN: 0.0410170815885067

P-Value for LEVENE: 0.05070013301303011


>P-Value for Shapiro Wilk test is < 0.05, so we should use Kruskal Wallis Test

In [110]:

stat, p_value = kruskal(data["BICYCLE COLLISIONS"], data["MOTORCYCLE COLLISIONS"], data["PEDESTRIAN COLLISIONS"])
print(f"Kruskal–Wallis H = {stat:.3f}, p = {p_value:.5f}")

Kruskal–Wallis H = 18.114, p = 0.00012


> Conslusion: P-Value for Kruskal Wallis test is < 0.05, at least one group's mean differs significantly

In [111]:
# Tukeys HSD:

values = np.concatenate([data["BICYCLE COLLISIONS"], data["MOTORCYCLE COLLISIONS"], data["PEDESTRIAN COLLISIONS"]])
groups = (['bike'] * len(data["BICYCLE COLLISIONS"]) +
          ['moto'] * len(data["MOTORCYCLE COLLISIONS"]) +
          ['ped']  * len(data["PEDESTRIAN COLLISIONS"]))

df = pd.DataFrame({'value': values, 'group': groups})

tukey = pairwise_tukeyhsd(endog=df['value'], groups=df['group'], alpha=0.05)
tukey.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
bike,moto,-2.2857,0.9886,-40.9223,36.3508,False
bike,ped,98.6429,0.0,60.0063,137.2794,True
moto,ped,100.9286,0.0,62.292,139.5651,True


> Conclusion: The mean collision counts for bicycle and motorcycle groups do not differ significantly; only the pedestrian group’s mean is significantly higher.

### Do fatalities differ for collisions occuring mid-Block and at an intersection?

In [112]:
# Testing Normaliry using Shapiro-Wilk

difference = data["FATALITIES: MIDBLOCK"] - data["FATALITIES: INTERSECTION"]
_, p_value = shapiro(difference)

print("P-Value for Shapiro Wilk Test: " + str(p_value))

P-Value for Shapiro Wilk Test: 0.014968442730605602


> P-Value is < 0.05. Not normal, so use Wilcoxin signed-rank

In [113]:
stat, p_value = wilcoxon(difference)
print(f"Wilcoxon W = {stat:.3f}, p = {p_value:.5f}")

Wilcoxon W = 24.000, p = 0.23659


c:\MADE\Program Files\Python\lib\site-packages\scipy\stats\_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


> Conslusion: P-Value for Wilcoxon is > 0.05, there is no significant difference in fatalities occuring midblock and at an intersection

### Do injuries differ for collisions occuring mid-Block and at an intersection?

In [114]:
# Testing Normaliry using Shapiro-Wilk

difference_intersection = data["INJURIES: MIDBLOCK"] - data["INJURIES: INTERSECTION"]
_, p_value = shapiro(difference_intersection)

print("P-Value for Shapiro Wilk Test: " + str(p_value))

P-Value for Shapiro Wilk Test: 0.0007438326138071716


> P-Value is < 0.05. Not normal, so use Wilcoxin signed-rank

In [115]:
stat, p_value = wilcoxon(difference_intersection)
print(f"Wilcoxon W = {stat:.3f}, p = {p_value:.5f}")

Wilcoxon W = 0.000, p = 0.00012


In [116]:
mean_int  = data["INJURIES: INTERSECTION"].mean()
mean_mid  = data["INJURIES: MIDBLOCK"].mean()

print(f"Mean Intersection Injuries: {mean_int:.1f}")
print(f"Mean Midblock Injuries:    {mean_mid:.1f}")

Mean Intersection Injuries: 2886.4
Mean Midblock Injuries:    1073.4


> Conclusion: Since p < 0.05, injuries differ significantly between midblock and intersection collisions. On average, intersection collisions result in far more injuries (2886.4) than midblock collisions (1073.4).

# Inference Testing II

### Do weather conditions significantly improve prediction of collision counts beyond traffic volume?

* H_0 = Weather data adds not value beyond traffic volume -> β_precip = β_temp = β_vis = 0
* H_α = Weather data adds value beyond trafficn volume

In [117]:
# Loading relevant columns
y_col   = "Total Collisions"
vol_col = "Yearly Average of Daily Volume"
w_cols  = ["Average temperature (degrees C)",
           "Average wind Speed (km/h)",
           "Average visibility (km)"]

df = data[[y_col, vol_col] + w_cols].dropna().copy()
y = df[y_col].to_numpy(float)
df

,Total Collisions,Yearly Average of Daily Volume,Average temperature (degrees C),Average wind Speed (km/h),Average visibility (km)
1,23442,12193.134872,3.445416,13.142069,16.600753
2,23243,11896.012759,4.414667,12.638326,16.167048
3,24805,11975.502906,2.410434,12.647422,18.062787
4,24627,12330.219780,2.810970,12.625795,18.412174
5,25516,13160.213272,4.941236,12.332064,18.046739
6,23139,12688.032454,4.812860,12.175093,17.538099
7,23905,11896.841582,3.731350,12.627638,17.347945
8,24003,11064.122497,1.785195,12.315178,17.535189
9,21943,11392.131920,2.479827,12.075129,18.848681
10,15804,9784.876325,3.019966,13.090865,19.538691


In [118]:
# Helper functions:

def add_intercept(X): return np.c_[np.ones(len(X)), X]

def ols_rss(X, y):
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)
    resid = y - X @ beta
    RSS = float(resid.T @ resid)
    dof = len(y) - X.shape[1]
    return RSS, dof

In [119]:
# Restricted model (traffic volume only)
Xr = add_intercept(df[[vol_col]].to_numpy(float))
RSS_r, dof_r = ols_rss(Xr, y)

In [120]:
# Full model (traffic Volume + Weather)
Xf = add_intercept(df[[vol_col] + w_cols].to_numpy(float))
RSS_f, dof_f = ols_rss(Xf, y)

In [121]:
# Partial F Test
q = Xf.shape[1] - Xr.shape[1]
F_stat = ((RSS_r - RSS_f)/q) / (RSS_f/dof_f)
p_val  = stats.f.sf(F_stat, q, dof_f)

print(f"Weather block partial F-test: F({q}, {dof_f}) = {F_stat:.3f}, p = {p_val:.4g}")

Weather block partial F-test: F(3, 7) = 5.115, p = 0.03481


> Conclusion: Since p < 0.05, we reject H_0. We now know that weather adds significant predictive value beyond traffic volume.

### Did the mean collision risk (Total Collisions ÷ Yearly Average of Daily Volume) differ between 2010–2019 (pre-pandemic) and 2020–2023 (post-pandemic)?

* H_0: mean pre-covid = mean post-covid
* H_α: mean pre-covid ≠ mean post-covid

In [122]:
# Data:
df = data.copy()
df["risk"] = data["Total Collisions"] / data["Yearly Average of Daily Volume"]
df["risk"]

0          NaN
1     1.922557
2     1.953848
3     2.071312
4     1.997288
5     1.938874
6     1.823687
7     2.009357
8     2.169445
9     1.926154
10    1.615146
11    1.650560
12    1.834477
13         NaN
Name: risk, dtype: float64

In [123]:
# Split time periods:
pre = df.loc[(df["Year"] >= 2010) & (df["Year"] <= 2019), "risk"].dropna()
post = df.loc[(df["Year"] >= 2020) & (df["Year"] <= 2023), "risk"].dropna()

In [124]:
# Normality check (Shapiro–Wilk) for guidance
def shapiro_p(x):
    return np.nan if len(x) < 3 else stats.shapiro(x).pvalue

p_pre, p_post = shapiro_p(pre), shapiro_p(post)

print(f"P-Value from Shapiro Wilk Test - Pre Covid Era: {p_pre:.3f}")
print(f"P-Value from Shapiro Wilk Test - Post Covid Era: {p_post:.3f} ")

P-Value from Shapiro Wilk Test - Pre Covid Era: 0.761
P-Value from Shapiro Wilk Test - Post Covid Era: 0.288 


In [125]:
# Welch two-sample t-test (unequal variances)
welch = stats.ttest_ind(pre, post, equal_var=False)

print(f"P-Value from Welch's T-test: {welch.pvalue:.3f}")
print(f"Pre (2010–2019): n={len(pre)}, mean risk={pre.mean():.6f}")
print(f"Post (2020–2023): n={len(post)}, mean risk={post.mean():.6f}\n")

P-Value from Welch's T-test: 0.034
Pre (2010–2019): n=9, mean risk=1.979169
Post (2020–2023): n=3, mean risk=1.700061



> Conclusion: Since the P-value for two-sample t-test is 0.034, which is less than 0.05 - we reject the null hypothesis. This means that mean collision risk changed between periods - it was lower post-2020 than pre-2020.

### Do the yearly fatality proportions (fatalities ÷ [fatalities + injuries]) differ between intersection and mid-block collisions?

* H0: mean(p_int − p_mid) = 0
* H1: mean(p_int − p_mid) ≠ 0

In [ ]:
# Columns aliases

f_int, i_int = "FATALITIES: INTERSECTION", "INJURIES: INTERSECTION"
f_mid, i_mid = "FATALITIES: MIDBLOCK",     "INJURIES: MIDBLOCK"

In [132]:
# Yearly severity proportions

df = data.copy()
den_int = df[f_int] + df[i_int]
den_mid = df[f_mid] + df[i_mid]
mask = den_int.gt(0) & den_mid.gt(0)

p_int = (df.loc[mask, f_int] / den_int[mask]).rename("p_int") #proportions
p_mid = (df.loc[mask, f_mid] / den_mid[mask]).rename("p_mid")
years = df.loc[mask, "Year"].astype(int).to_numpy()
diffs = (p_int - p_mid).to_numpy()

diffs

array([-0.00669061, -0.00314374, -0.01634104, -0.00665686, -0.0085276 ,
       -0.0069702 , -0.00806656, -0.0101657 , -0.00934238, -0.00529986,
       -0.01994751, -0.01474539, -0.00605956, -0.00225088])

In [137]:
# Normality of paired differences
shapiro_stat, shapiro_p = stats.shapiro(diffs)
print(f"Number of years = {len(diffs)}")
print(f"Mean p_int = {p_int.mean():.4f}, Mean p_mid = {p_mid.mean():.4f}")
print(f"Shapiro-Wilk p-value (diffs): {shapiro_p:.4f}")

Number of years = 14
Mean p_int = 0.0036, Mean p_mid = 0.0125
Shapiro-Wilk p-value (diffs): 0.1557


> Shapiro Wilk test's p-value = 0.1557, which is greater than 0.05. We fail to reject normality of paired differences. Therefore, we can conclude that the data is consistent with normality - we can use Paired-T test.

In [138]:
t_stat, t_p = stats.ttest_rel(p_int, p_mid)
n = len(diffs)
md = diffs.mean()
sd = diffs.std(ddof=1)
ci_halfwidth = stats.t.ppf(0.975, df=n-1) * sd / np.sqrt(n)
print("\nPaired t-test (two-sided)")
print(f"t = {t_stat:.3f}, p = {t_p:.4f}")
print(f"Mean diff (p_int - p_mid) = {md:.4f}")
print(f"95% CI for mean diff = [{md-ci_halfwidth:.4f}, {md+ci_halfwidth:.4f}]")


Paired t-test (two-sided)
t = -6.627, p = 0.0000
Mean diff (p_int - p_mid) = -0.0089
95% CI for mean diff = [-0.0118, -0.0060]


> Conclusion: p-value < 0.001 - there is strong evidence against the null hypothesis. Therefore we reject the null hypothesis, and state that there is statistically significant difference inseverity mix by location.